In [1]:
# library import 
import torch
from torch import nn
import numpy as np
import os
import csv
import pickle
import os
import random
from PIL import Image
import torch.nn.functional as F

In [47]:
def read_character_stat(description_path):
    info = {}
    with open(description_path) as csv_file:
        csv_lines = csv.DictReader(csv_file)
        for row in csv_lines:
            image = row['image']
            label = ord(row['label'])-ord('0')
            if info.get(label) is None:
                print(row['label'], label)
                info[label] = []
            info[label].append(image)
    return info

In [3]:
def division_train_test(image_file_path, description_path):
    info = read_character_stat(description_path)
    train, test = 50, 5
    division = {}
    for key in info:
        random.shuffle(info[key])
        division[key]={}
        division[key]['train'] = []
        division[key]['test'] = [] 
        for i in range(0, test):
            division[key]['test'].append(info[key][i])
        for i in range(test, len(info[key])):
            division[key]['train'].append(info[key][i])
        # print(key, type(key),len(division[key]['train']), len(division[key]['test']))
    return division

In [48]:
image_file_path = os.path.join('.','datasets','Img')
description_path = os.path.join('.','datasets','english.csv')
division = division_train_test(image_file_path, description_path)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
A 17
B 18
C 19
D 20
E 21
F 22
G 23
H 24
I 25
J 26
K 27
L 28
M 29
N 30
O 31
P 32
Q 33
R 34
S 35
T 36
U 37
V 38
W 39
X 40
Y 41
Z 42
a 49
b 50
c 51
d 52
e 53
f 54
g 55
h 56
i 57
j 58
k 59
l 60
m 61
n 62
o 63
p 64
q 65
r 66
s 67
t 68
u 69
v 70
w 71
x 72
y 73
z 74


In [6]:
def find_unique_values(np_arr):
    return np.unique(np_arr)

In [7]:
# load and show an image with Pillow
def read_image_file(file_name):
    # Open the image form working directory
    image = Image.open(os.path.join('.', 'datasets', file_name))
    # summarize some details about the image
    print(image.format)
    print(image.size)
    print(image.mode)
    print(type(image))
    new_size = (30, 40)
    image = image.resize(new_size)
    # convert image to numpy array
    data = np.asarray(image)
    print(type(data))
    # summarize shape
    print(data.shape)
    #data = np.where(data==255, 1, data) 
    # print(find_unique_values(data))
    # print(data[0:5])
    #reduced_data = data[:, :, :1]
    #reduced_data = reduced_data.reshape(30,40)
    reduced_data = data
    reduced_data = reduced_data.transpose(2,1,0)
    print(find_unique_values(reduced_data))
    return reduced_data

In [8]:
data = read_image_file(division[0]['train'][0])
print(data.shape)
print(data)

PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   3   5   6   7   8   9  10  11  12  14  16  18  22  23  25  26
  27  31  42  43  47  48  57  64  66  71  78  81  87  88  90  95  97  99
 101 102 103 106 111 114 116 120 121 127 130 131 133 136 137 139 141 146
 150 154 156 159 165 167 168 169 176 179 181 189 190 191 193 195 202 203
 204 205 207 210 213 214 215 220 221 222 224 227 233 237 242 243 246 247
 249 251 252 253 254 255]
(3, 30, 40)
[[[255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  ...
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]]

 [[255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  ...
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]]

 [[255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  .

In [52]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 20, 3)
        
        self.fc1 = nn.Linear(12 * 7 * 20, 120)
        self.fc2 = nn.Linear(120, 75)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [11]:
model = Model()
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 1
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [51]:
P_X = []
P_Y = []
t = [0, 1, 2]
for key in division:
    for j in range(0, len(division[key]['train'])):
        f_n = division[key]['train'][j]
        x = read_image_file(f_n)
        P_X.append(x)
        P_Y.append(key)
print(len(P_X))
print(P_X[0].shape)

PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   3   4  24  25  26  30  44  47  53  56  57  73  86  90 101 102
 128 138 144 145 148 152 153 154 156 157 165 169 176 181 185 187 189 195
 215 225 238 244 246 247 248 250 251 252 253 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   2   3   8   9  10  12  15  16  22  24  32  34  39  42  47  48
  51  52  54  60  62  66  77  80 100 103 110 111 114 119 124 127 130 133
 136 148 149 154 158 159 160 162 164 168 179 180 181 188 194 197 201 206
 212 213 214 220 221 225 232 233 235 238 242 244 245 248 250 251 253 254
 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   5  11  13  14  35  39  40  44  47  50  55  56  57  76  78  85
  90  97  98 102 106 107 118 122 128 133 148 164 176 186 191 194 202 206
 208 212 221 223 225 247 248 251 252 253 254 255]
PNG
(1200, 900)


<class 'numpy.ndarray'>
(40, 30, 3)
[  0   2   4   6   7   8  10  11  13  15  16  17  21  24  28  31  32  35
  38  45  49  51  53  58  61  62  69  71  77  82  84  92  96 100 101 102
 112 115 117 118 119 121 122 134 138 144 150 160 161 162 165 167 174 178
 189 191 193 195 198 199 200 206 207 213 217 219 220 226 227 228 231 234
 236 238 240 242 244 248 249 250 252 253 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   3   5   6   7   8   9  10  11  12  14  15  16  17  18  20  21
  23  24  30  31  32  39  40  44  49  53  54  58  61  62  63  71  72  74
  80  82  83  86  93  96 103 105 107 108 110 111 113 114 117 120 123 129
 136 143 145 148 151 154 158 166 167 170 176 178 180 182 183 184 185 187
 188 189 192 196 197 200 201 208 210 213 214 216 220 221 227 228 229 230
 231 234 235 237 241 242 243 244 248 250 251 252 253 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)

<class 'numpy.ndarray'>
(40, 30, 3)
[  0   2  10  14  19  23  38  52  63  66  80 115 120 129 135 140 144 152
 153 158 162 166 167 176 178 185 187 192 194 195 201 206 208 209 214 216
 219 222 229 232 237 248 250 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   3   4   9  13  20  21  23  27  40  45  53  73  81  83  93  94  96
 102 108 111 119 123 135 136 139 143 148 151 153 154 161 162 167 168 169
 170 176 179 180 181 182 187 190 194 196 200 209 210 218 219 227 232 234
 236 240 241 246 248 251 253 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   7  10  22  28  32  33  34  35  38  39  41  42  44  45  46  50  53
  54  56  57  61  71  74  83  98 110 145 159 171 191 201 206 213 214 234
 236 243 246 247 249 250 251 252 253 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0  26  31  45  52  56  59  61  62  67 

PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   2   3   4   5   7  11  13  14  17  19  20  21  23  24  26  28  30
  32  33  34  35  36  43  45  50  53  63  64  65  71  84  99 102 105 109
 132 137 138 142 150 152 153 156 160 167 170 174 175 178 184 185 186 188
 198 200 201 202 211 214 215 220 224 227 229 231 233 237 242 245 246 250
 251 252 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   3   5   7   9  13  15  16  27  28  29  32  36  37  51  53  59
  61  63  64  72  74  75  79  80  86  89  90  96 105 107 114 121 124 125
 127 129 137 139 144 148 159 160 161 162 165 166 170 172 173 181 190 192
 193 201 211 214 215 224 225 228 229 230 236 238 240 244 245 247 248 252
 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   2   4   5   8  10  14  19  20  22  29  31  33  38  39  44  46
  51  56  64  65  66  71 

<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   3   4   5   6   8  11  17  19  24  25  26  27  28  31  34  42
  43  44  46  49  55  56  58  61  64  81  84  85  91  95  97 104 106 109
 110 112 116 127 128 129 139 141 150 153 154 158 159 162 164 165 166 169
 170 175 177 178 179 180 187 190 191 192 194 195 198 199 202 210 219 226
 227 231 232 234 235 238 240 241 243 247 248 249 252 253 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   3   4   7   8  13  14  18  20  22  29  33  34  35  45  52  73  83
  89  97 111 126 128 141 144 158 161 163 175 179 180 181 183 193 202 206
 210 211 218 224 226 236 238 240 241 248 251 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   2   5   6   7   9  10  11  13  18  27  34  40  49  52  57  59
  60  62  78  85  89  95  98 110 113 116 117 123 126 128 133 138 139 152
 160 165 166 167 168 169 186 193 195 198 205 207 208 210 211 212 213

...

<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   2   3   4   5  10  13  16  17  18  19  20  21  23  25  26  34
  37  40  44  55  59  61  65  72  79  88  95 100 105 107 110 111 114 120
 129 136 143 151 152 156 162 169 170 180 181 189 191 198 201 203 210 213
 225 227 230 233 237 240 242 246 247 248 250 252 253 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   4   5   7   8   9  11  19  23  24  27  28  29  33  36  38  44
  48  49  53  54  56  58  59  66  69  79  82  83  85  86  93  98  99 107
 119 120 121 122 123 140 149 150 154 163 167 177 185 190 193 195 200 201
 203 206 208 213 214 215 217 221 229 230 231 233 236 238 241 242 243 244
 245 247 248 249 250 251 252 253 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   2   3   5   8  10  16  18  26  31  35  36  38  39  40  46  56  66
  72  77  86  89  99 106 121 123 128 132 133 137 143 145 166 173 175 178
 185 186 19

<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   2   4   5   9  10  11  20  24  38  40  45  47  48  50  52  53
  62  64  67  72  76  79  83  84  96 101 105 120 125 134 137 143 146 148
 153 155 165 168 170 171 182 193 196 202 204 207 214 216 218 219 220 223
 226 235 249 250 251 252 253 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   2   5  11  14  15  16  21  24  25  28  30  37  39  42  43  45
  46  62  67  73  77  87  89  91  94 101 102 103 108 112 114 120 129 161
 165 171 172 173 174 176 186 189 197 198 218 224 226 228 230 233 235 237
 238 241 247 248 249 250 251 252 253 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   3   5   9  12  13  14  16  20  29  34  45  50  72  76  80  81  87
  88  90 101 110 112 118 136 144 146 148 151 159 165 167 183 197 206 214
 216 218 223 224 230 234 235 237 244 245 253 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImag

<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   4   9  10  12  13  15  18  20  22  26  27  28  30  31  41  46
  62  68  73  75  78  85  86  88  89  98 100 114 117 120 121 130 135 138
 142 145 157 161 163 164 165 166 168 177 179 180 181 188 190 192 194 198
 216 218 219 225 226 227 230 232 233 234 238 241 243 246 248 250 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   2   3   6   7   8  16  17  23  27  30  31  32  33  40  42  47
  48  62  63  64  74  80  89  97 103 105 106 126 127 133 140 146 158 159
 161 163 175 178 180 182 186 189 203 208 215 216 224 225 230 232 234 237
 238 241 242 243 246 250 251 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   3   6  11  12  14  19  21  22  29  33  47  53  59  63  64  87
  94  95  99 103 104 110 112 114 130 131 134 135 137 145 150 154 158 162
 164 168 174 176 177 179 182 187 191 192 193 198 203 205 211 212 218 223

<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   3   4   5   6   7   8  11  14  16  19  20  22  25  31  34  35
  36  37  38  39  41  43  44  45  47  50  51  55  60  61  62  63  64  67
  69  70  80  81  82  86  88  89  96 102 105 108 109 112 124 133 138 139
 143 148 152 165 168 174 176 178 179 184 188 191 192 200 204 206 210 214
 219 222 223 224 226 232 233 237 238 242 243 244 247 248 250 251 252 253
 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   2   5   7  16  17  18  22  27  36  43  46  48  49  55  56  62
  67  72  73  76  83  85  89  90  98 101 105 111 114 131 133 136 139 142
 144 146 151 167 168 173 178 182 188 190 194 196 198 209 213 218 219 220
 221 222 225 230 231 233 237 238 241 242 243 245 246 247 254 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   2   3   6   8   9  11  13  14  15  18  20  26  28  29  30  40
  43  47  48  49  54  61  65  66  7

<class 'numpy.ndarray'>
(40, 30, 3)
[  0   3  16  22  27  29  38  40  43  48  52  55  56  57  61  66  76  93
 100 102 109 123 128 131 134 143 146 149 152 155 158 168 170 172 178 187
 188 190 191 193 198 203 208 223 230 237 239 241 243 244 245 249 251 252
 253 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   1   2   3   4   5   7   9  12  13  14  15  18  19  21  27  31  32
  34  42  44  47  52  53  57  59  60  71  72  75  78  81  84  89  93 106
 116 117 120 124 134 140 145 155 157 158 161 162 163 165 168 176 185 188
 189 192 197 198 206 207 212 215 219 222 223 225 227 229 233 235 237 239
 244 246 249 252 253 255]
PNG
(1200, 900)
RGB
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>
(40, 30, 3)
[  0   6   7  10  11  12  15  24  27  42  45  57  70  71  76  79  85  86
  88  90  98 110 126 128 138 141 145 147 150 152 157 172 175 184 194 209
 212 214 215 216 217 233 235 242 248 250 252 253 254 255]
PNG
(1200, 900)
R

In [13]:
P_X = torch.Tensor(P_X)
P_Y = torch.Tensor(P_Y)

In [14]:
print(P_X.shape)
print(P_Y.shape)

torch.Size([6, 3, 30, 40])
torch.Size([6])


In [15]:
def save_chekcpoint(epoch, model_path, loss):
    EPOCH = epoch
    PATH = model_path
    LOSS = loss

    torch.save({
                'epoch': EPOCH,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': LOSS,
                }, PATH)

In [16]:
def load_checkpoint(PATH):
    if os.path.exists(PATH) is True:
        model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=20, n_layers=1)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        checkpoint = torch.load(PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print("checkpoint loaded")
        print(checkpoint['model_state_dict'])
        criterion = nn.CrossEntropyLoss()
        return epoch, loss, model, optimizer
    else:
        return None, None, None, None

In [17]:
already_loss = None

In [18]:
already_epoch, already_loss, model_, optimizer_ = load_checkpoint('model.pt')
if model_ is not None:
    model = model_
    optimizer = optimizer_
    model.train()
print(already_epoch, already_loss)

None None


In [19]:
# Training Run
P_X = P_X.to(device)
P_Y = P_Y.to(device)

In [23]:
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    #input_seq = input_seq.to(device)
    outputs = model(P_X)
    outputs = outputs.to(device)
    # print(outputs)
    loss = criterion(outputs, P_Y.long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 1:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 1/1............. Loss: 0.7054


In [42]:
if already_loss is not None and loss.item() < already_loss:
    save_chekcpoint(already_epoch+n_epochs, 'model.pt', loss.item())
if already_loss is None:
    already_loss, already_epoch = loss.item(), n_epochs
    save_chekcpoint(already_epoch, 'model.pt', loss.item())

In [38]:
def prediction(file_name):
    model.eval() # eval mode
    x = [read_image_file(file_name)]
    x = torch.Tensor(x)
    x = x.to(device)
    print(x.shape)
    output = model(x)
    print(output,shape)
    prob = nn.functional.softmax(output[-1], dim=0).data
    idx = torch.max(prob, dim=0)[1].item()
    # print(prob, idx)
    return idx

In [49]:
def prediction_initiate():
    result = {}
    for key in division:
        char_value = chr(key + 48)
        result[char_value] = 0
        for i in range(0, len(division[key]['test'])):
            file_name = division[key]['train'][j]
            idx = prediction(file_name)
            if idx == key:
                result[char_value] = res[char_value] + 1
        result[char_value] = (result[char_value] * 100.0 / (1.0 * len(division[key]['test'])))
    return result

In [50]:
result = prediction_initiate()
print(result)

0
1
2
3
4
5
6
7
8
9
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z
